#### GUI 구성

In [2]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import os

### 기준정보로 등록해야 하는 정보
## GUI 타이틀
win_title = "Image Judge"

## GUI 크기
Gsize = "630x200"

## 폴더 / 파일 리스트
# 각 행의 라벨 정보 List로 정리
label_f = ['Target Folder', 'Result Folder']
# 각 행에서 다루는 값이 폴더일때는 0, 파일일때는 1로 구분자
fileyn = [0, 0]
# 폴더 / 파일 리스트 기준정보 불러오기 (기준정보 관리 파일명 : GUI_Master.csv)
# 기준정보 파일이 없을 경우 초기화
try : 
    df_FileFolder= pd.read_csv("GUI_Master.csv")
except :
    d = {'Item' : ['Please select file / folder path'] * len(label_f)}
    df_FileFolder = pd.DataFrame(data=d)

## 옵션 리스트
# 각 행의 라벨 정보 List로 정리
label_option = []

## 파라미터 텍스트 상자 리스트
label_para = ['영역 기준', '판정 기준']
# 폴더 / 파일 리스트 기준정보 불러오기 (기준정보 관리 파일명 : Para.csv)
# 기준정보 파일이 없을 경우 초기화
try : 
    dfP= pd.read_csv("Para.csv")
except :
    d = {'Item' : ['Input parameter'] * len(label_para)}
    dfP = pd.DataFrame(data=d)

### 주요 함수
## 개발된 함수 추가
def func1():
    update_Para()
    pass

### GUI용 함수
# GUIMaster Data 업데이트 : 
def update_Master(idx, var):
    df_FileFolder.Item[idx] = var
    df_FileFolder.to_csv('GUI_Master.csv', index=False)

# Para Data 업데이트 :
def update_Para():
    for i in range(len(label_para)):
        dfP.Item[i] = txtPara[i].get("1.0",tk.END)
    dfP.to_csv('Para.csv', index=False)

# 폴더/파일 경로 바꾸는 버튼을 눌렀을때 업데이트
def onClick(i, fileYN):
    # 폴더 경로 바꾸는 로직 (fineYN = 0 일때)
    if fileYN == 0:
        folder_selected = filedialog.askdirectory()
        var = folder_selected
    # 파일 경로 바꾸는 로직 (fineYN = 0 이 아닐때)
    else:
        folder_selected = filedialog.askopenfile()
        var = folder_selected.name

    txtPath[i].delete('1.0', tk.END)
    txtPath[i].insert(tk.INSERT, chars=var)
    update_Master(i,var)
    
## Main Code
# GUI 구성
win = tk.Tk()
win.geometry(Gsize)
win.title(win_title)

# Frame 설정하기
frameF = tk.LabelFrame(win, pady=5, padx = 5, text = 'Folder / File Path')
frameF.pack()
frameR = tk.LabelFrame(win, pady=5, padx = 5, text='Option')
frameR.pack()
frameP = tk.LabelFrame(win, pady=5, padx = 5, text = 'Parameter')
frameP.pack()
frameB = tk.Frame(win, pady=5, padx = 5)
frameB.pack()

# 파일 / 폴더 경로 설정 GUI
lbName = []
txtPath = []
btnPath =[]

for i,x in enumerate(label_f):
    lbName.append(tk.Label(frameF, text=x, width=15,padx =5, pady = 5))
    txtPath.append(tk.Text(frameF, width = 50, height = 1, padx =5, pady = 5, background='lightgrey'))
    btnPath.append(tk.Button(frameF, text="Change Path", width=10, padx =5, pady = 5, command=lambda i=i: onClick(i,fileyn[i])))

    # 폴더/파일 이름 초기값 넣기
    txtPath[i].insert(tk.INSERT, chars=df_FileFolder.Item[i])

    lbName[i].grid(row=i, column=0, padx =5, sticky=tk.W)
    txtPath[i].grid(row=i, column=1, padx =5, sticky=tk.W)
    btnPath[i].grid(row=i, column=2, padx =5, sticky=tk.W)

# Radio 상자 GUI
option_var = tk.IntVar()
option_var.set(0)
option_radio = []

for i,x in enumerate(label_option):
    option_radio.append(tk.Radiobutton(frameR, text=x, variable=option_var, width = 24, padx=3, value=i))
    option_radio[i].grid(row=i // 3, column=i % 3, sticky=tk.W)

# Parameter 설정 GUI
lbPara = []
txtPara = []

for i,x in enumerate(label_para):
    lbPara.append(tk.Label(frameP, text=x, width=15, padx=2))
    txtPara.append(tk.Text(frameP, width = 25, height = 1, padx =5, pady = 5))
    lbPara[i].grid(row=((i) // 2), column = ((i) % 2 ) * 2 , sticky=tk.W)
    txtPara[i].grid(row=((i) // 2), column=((i) % 2 ) * 2 + 1, sticky=tk.W)

    # 파라미터 초기값 넣기
    txtPara[i].insert(tk.INSERT, chars=dfP.Item[i])
    

# 버튼 설정 GUI
btnMergeXl = tk.Button(frameB, text = '기능 버튼', width = 35, padx=5, pady=5, command = update_Para)
btnMergeXl.pack()

win.mainloop()

#### AI 코딩 내용 함수로 전환하기

In [ ]:
import cv2
import numpy as np

# define image processing function
def image_judge():
    update_Para()
    target_folder = df_FileFolder.Item[0]
    result_folder = df_FileFolder.Item[1]
    area_standard = int(txtPara[0].get("1.0",tk.END))
    judge_standard = int(txtPara[1].get("1.0",tk.END))

    files = os.listdir(target_folder)
    
    for file in files:
        # Step 1: Load the image
        image_path = os.path.join(target_folder, file)  # Update with your image path
        NG_image_path = os.path.join(result_folder, file)  
        image = cv2.imread(image_path)

        # Step 2: Apply a blur effect to the image
        blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

        # Step 3: Convert the blurred image to grayscale
        gray_blurred_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)

        # Step 4: Threshold the grayscale image to separate black and white regions
        _, binary_image = cv2.threshold(gray_blurred_image, area_standard, 255, cv2.THRESH_BINARY)

        # Step 5: Identify the black regions in the thresholded image
        black_region = binary_image == 0

        # Step 6: Check the brightness of the corresponding pixels in the original image
        # Convert the original image to grayscale
        gray_original_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Step 7: Check if any pixel in the black region has brightness > 10
        if np.any(gray_original_image[black_region] > judge_standard):
            # Save the image to the result folder
            cv2.imwrite(NG_image_path, image)


#### 코드 합쳐서 최종 코드 만들기

In [6]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import os
import cv2
import numpy as np

### 기준정보로 등록해야 하는 정보
## GUI 타이틀
win_title = "Image Judge"

## GUI 크기
Gsize = "630x200"

## 폴더 / 파일 리스트
# 각 행의 라벨 정보 List로 정리
label_f = ['Target Folder', 'Result Folder']
# 각 행에서 다루는 값이 폴더일때는 0, 파일일때는 1로 구분자
fileyn = [0, 0]
# 폴더 / 파일 리스트 기준정보 불러오기 (기준정보 관리 파일명 : GUI_Master.csv)
# 기준정보 파일이 없을 경우 초기화
try : 
    df_FileFolder= pd.read_csv("GUI_Master.csv")
except :
    d = {'Item' : ['Please select file / folder path'] * len(label_f)}
    df_FileFolder = pd.DataFrame(data=d)

## 옵션 리스트
# 각 행의 라벨 정보 List로 정리
label_option = []

## 파라미터 텍스트 상자 리스트
label_para = ['영역 기준', '판정 기준']
# 폴더 / 파일 리스트 기준정보 불러오기 (기준정보 관리 파일명 : Para.csv)
# 기준정보 파일이 없을 경우 초기화
try : 
    dfP= pd.read_csv("Para.csv")
except :
    d = {'Item' : ['Input parameter'] * len(label_para)}
    dfP = pd.DataFrame(data=d)

### 주요 함수
## 개발된 함수 추가
# define image processing function
def image_judge():
    update_Para()
    target_folder = df_FileFolder.Item[0]
    result_folder = df_FileFolder.Item[1]
    area_standard = int(txtPara[0].get("1.0",tk.END))
    judge_standard = int(txtPara[1].get("1.0",tk.END))

    files = os.listdir(target_folder)
    
    for file in files:
        # Step 1: Load the image
        image_path = os.path.join(target_folder, file)  # Update with your image path
        NG_image_path = os.path.join(result_folder, file)  
        image = cv2.imread(image_path)

        # Step 2: Apply a blur effect to the image
        blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

        # Step 3: Convert the blurred image to grayscale
        gray_blurred_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)

        # Step 4: Threshold the grayscale image to separate black and white regions
        _, binary_image = cv2.threshold(gray_blurred_image, area_standard, 255, cv2.THRESH_BINARY)

        # Step 5: Identify the black regions in the thresholded image
        black_region = binary_image == 0

        # Step 6: Check the brightness of the corresponding pixels in the original image
        # Convert the original image to grayscale
        gray_original_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Step 7: Check if any pixel in the black region has brightness > 10
        if np.any(gray_original_image[black_region] > judge_standard):
            # Save the image to the result folder
            cv2.imwrite(NG_image_path, image)

### GUI용 함수
# GUIMaster Data 업데이트 : 
def update_Master(idx, var):
    df_FileFolder.Item[idx] = var
    df_FileFolder.to_csv('GUI_Master.csv', index=False)

# Para Data 업데이트 :
def update_Para():
    for i in range(len(label_para)):
        dfP.Item[i] = txtPara[i].get("1.0",tk.END)
    dfP.to_csv('Para.csv', index=False)

# 폴더/파일 경로 바꾸는 버튼을 눌렀을때 업데이트
def onClick(i, fileYN):
    # 폴더 경로 바꾸는 로직 (fineYN = 0 일때)
    if fileYN == 0:
        folder_selected = filedialog.askdirectory()
        var = folder_selected
    # 파일 경로 바꾸는 로직 (fineYN = 0 이 아닐때)
    else:
        folder_selected = filedialog.askopenfile()
        var = folder_selected.name

    txtPath[i].delete('1.0', tk.END)
    txtPath[i].insert(tk.INSERT, chars=var)
    update_Master(i,var)
    
## Main Code
# GUI 구성
win = tk.Tk()
win.geometry(Gsize)
win.title(win_title)

# Frame 설정하기
frameF = tk.LabelFrame(win, pady=5, padx = 5, text = 'Folder / File Path')
frameF.pack()
frameR = tk.LabelFrame(win, pady=5, padx = 5, text='Option')
frameR.pack()
frameP = tk.LabelFrame(win, pady=5, padx = 5, text = 'Parameter')
frameP.pack()
frameB = tk.Frame(win, pady=5, padx = 5)
frameB.pack()

# 파일 / 폴더 경로 설정 GUI
lbName = []
txtPath = []
btnPath =[]

for i,x in enumerate(label_f):
    lbName.append(tk.Label(frameF, text=x, width=15,padx =5, pady = 5))
    txtPath.append(tk.Text(frameF, width = 50, height = 1, padx =5, pady = 5, background='lightgrey'))
    btnPath.append(tk.Button(frameF, text="Change Path", width=10, padx =5, pady = 5, command=lambda i=i: onClick(i,fileyn[i])))

    # 폴더/파일 이름 초기값 넣기
    txtPath[i].insert(tk.INSERT, chars=df_FileFolder.Item[i])

    lbName[i].grid(row=i, column=0, padx =5, sticky=tk.W)
    txtPath[i].grid(row=i, column=1, padx =5, sticky=tk.W)
    btnPath[i].grid(row=i, column=2, padx =5, sticky=tk.W)

# Radio 상자 GUI
option_var = tk.IntVar()
option_var.set(0)
option_radio = []

for i,x in enumerate(label_option):
    option_radio.append(tk.Radiobutton(frameR, text=x, variable=option_var, width = 24, padx=3, value=i))
    option_radio[i].grid(row=i // 3, column=i % 3, sticky=tk.W)

# Parameter 설정 GUI
lbPara = []
txtPara = []

for i,x in enumerate(label_para):
    lbPara.append(tk.Label(frameP, text=x, width=15, padx=2))
    txtPara.append(tk.Text(frameP, width = 25, height = 1, padx =5, pady = 5))
    lbPara[i].grid(row=((i) // 2), column = ((i) % 2 ) * 2 , sticky=tk.W)
    txtPara[i].grid(row=((i) // 2), column=((i) % 2 ) * 2 + 1, sticky=tk.W)

    # 파라미터 초기값 넣기
    txtPara[i].insert(tk.INSERT, chars=dfP.Item[i])
    

# 버튼 설정 GUI
btnMergeXl = tk.Button(frameB, text = '기능 버튼', width = 35, padx=5, pady=5, command = image_judge)
btnMergeXl.pack()

win.mainloop()